In [20]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import pandas_ta as ta  # ATR 계산용 (없으면 pip install pandas_ta)

# ============================================================
# [설정]
# ============================================================
BASE_DIR = Path("2025-11-27_Sniper")
# ★ 중요: 원본 데이터 경로를 정확히 지정하세요
ORIGINAL_DATA_PATH = Path("../eth_4hour.csv")  # 실행 위치에 있다고 가정

FEE_RATE = 0.0005
RISK_PER_TRADE = 0.02   # 2% Risk (공격적)
MAX_PYRAMIDING = 5      # 최대 5개 포지션 중첩
INITIAL_EQUITY = 1.0
MODELS = ["CatBoost", "XGBoost", "LightGBM", "RandomForest"]

# ============================================================
# [데이터 복원 로직]
# ============================================================
def prepare_original_data(csv_path):
    """
    원본 4시간봉 데이터를 읽고, 전체 기간에 대한 ATR을 미리 계산해둡니다.
    """
    if not csv_path.exists():
        raise FileNotFoundError(f"원본 데이터가 없습니다: {csv_path}")
        
    df = pd.read_csv(csv_path)
    
    # 컬럼명 통일 (소문자로, timestamp/date 처리)
    df.columns = df.columns.str.lower()
    
    # 날짜 컬럼 찾기 ('timestamp' or 'date')
    date_col = 'timestamp' if 'timestamp' in df.columns else 'date'
    df[date_col] = pd.to_datetime(df[date_col])
    
    # ATR 계산 (High, Low, Close 필요)
    # 컬럼명이 eth_high 등일 수 있으니 확인
    h = df.get('high', df.get('eth_high'))
    l = df.get('low', df.get('eth_low'))
    c = df.get('close', df.get('eth_close'))
    
    # ATR 84 (전략 기본값)
    df['atr_84'] = ta.atr(h, l, c, length=84)
    
    # 진입가(다음 봉 시가) 미리 만들기
    # 전략: 시그널 뜬 봉(t) -> 진입은 t+1 시가
    # open 컬럼(eth_open)을 shift(-1) 해서 'next_open'으로 저장
    o = df.get('open', df.get('eth_open'))
    df['next_open'] = o.shift(-1)
    
    # 필요한 컬럼만 리턴 (날짜, ATR, 다음시가)
    return df[[date_col, 'atr_84', 'next_open']].rename(columns={date_col: 'timestamp'})

def restore_and_merge(preds_df, orig_data_df, profit_mult, stop_mult):
    """
    predictions.csv에 원본의 ATR, 가격 정보를 병합하여 TP/SL 복원
    """
    df = preds_df.copy()
    if "timestamp" not in df.columns: return df
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    
    # 1. 날짜 기준 병합 (Left Join)
    df = pd.merge(df, orig_data_df, on="timestamp", how="left")
    
    # 2. 가격 복원 (real_entry_price가 없으면 next_open 사용)
    if "real_entry_price" not in df.columns:
        # next_open도 없으면(데이터 끝) close로 대체
        df["real_entry_price"] = df["next_open"].fillna(df["close"])
        
    # 3. TP/SL 복원
    if "take_profit_price" not in df.columns:
        # atr이 NaN이면(초반 데이터) 0 처리
        atr = df["atr_84"].fillna(0)
        entry = df["real_entry_price"]
        
        df["take_profit_price"] = entry + (atr * profit_mult)
        df["stop_loss_price"] = entry - (atr * stop_mult)
        
    return df

# ============================================================
# [공격적 피라미딩 백테스트]
# ============================================================
def backtest_aggressive(df, fee_rate, risk, max_pos):
    df = df.sort_values("timestamp").reset_index(drop=True)
    equity = INITIAL_EQUITY
    equity_curve = [INITIAL_EQUITY]
    positions = [] # {entry, tp, sl, size}
    
    for i, row in df.iterrows():
        # 현재가(진입가) = 이번 봉의 시가(복원된 real_entry_price)
        curr_price = row["real_entry_price"]
        
        # 1. 기존 포지션 관리
        active_pos = []
        for pos in positions:
            # 익절 체크
            if curr_price >= pos['tp']:
                pnl = (pos['tp'] - pos['entry']) * pos['size']
                fee = pos['tp'] * pos['size'] * fee_rate
                equity += pnl - fee
                continue
            # 손절 체크
            if curr_price <= pos['sl']:
                pnl = (pos['sl'] - pos['entry']) * pos['size']
                fee = pos['sl'] * pos['size'] * fee_rate
                equity += pnl - fee
                continue
            active_pos.append(pos)
        positions = active_pos
        
        # 2. 신규 진입 (피라미딩)
        sig = int(row.get("signal", 0))
        if sig == 1 and len(positions) < max_pos:
            entry = float(row["real_entry_price"])
            sl = float(row["stop_loss_price"])
            tp = float(row["take_profit_price"])
            
            # 유효한 가격인지 확인 (ATR 계산 안돼서 0인 경우 제외)
            if entry > 0 and sl > 0 and entry > sl:
                loss_per_coin = entry - sl
                loss_pct = loss_per_coin / entry
                
                # 리스크 관리: 자산의 risk% 만큼만 손실나게 세팅
                risk_amt = equity * risk
                pos_size = risk_amt / loss_per_coin # 진입 코인 수
                pos_amt = pos_size * entry # 진입 금액
                
                # 수수료 차감
                equity -= pos_amt * fee_rate
                positions.append({'entry': entry, 'tp': tp, 'sl': sl, 'size': pos_size})

        # 3. 자산 평가 (Mark-to-Market)
        floating_pnl = sum([(curr_price - p['entry']) * p['size'] for p in positions])
        equity_curve.append(equity + floating_pnl)

    # 최종 수익률 & MDD
    eq_series = pd.Series(equity_curve)
    if len(eq_series) == 0: return 0.0, 0.0
    
    ret = eq_series.iloc[-1] / eq_series.iloc[0] - 1.0
    mdd = (eq_series / eq_series.cummax() - 1.0).min()
    
    return ret, mdd

# ============================================================
# [Main]
# ============================================================
if __name__ == "__main__":
    print("1. Loading Original Data (eth_4hour.csv)...")
    try:
        orig_data = prepare_original_data(ORIGINAL_DATA_PATH)
        print("   -> Success! ATR & Next_Open calculated.")
    except Exception as e:
        print(f"   -> Error: {e}")
        exit()

    results = []
    print("\n2. Starting Aggressive Backtest (All Trials)...")
    print(f"   Risk: {RISK_PER_TRADE*100}%, Max Pos: {MAX_PYRAMIDING}")

    for trial in sorted(BASE_DIR.iterdir()):
        if not trial.is_dir() or not trial.name.startswith("T"): continue
        
        # 파라미터 로드
        try:
            with open(next(trial.glob("*/metadata.json"))) as f: p = json.load(f).get("trial_params", {})
            p_mult, s_mult = float(p.get("profit_mult", 1.6)), float(p.get("stop_mult", 0.5))
        except: continue

        print(f"   Processing {trial.name}...")
        
        for model in MODELS:
            # Prediction 파일 모으기
            dfs = []
            for sub in trial.iterdir():
                if not sub.is_dir(): continue
                path = sub / f"predictions_{model}.csv"
                if path.exists():
                    d = pd.read_csv(path)
                    with open(sub/"metadata.json") as f: meta = json.load(f)
                    d["fold_type"] = meta.get("fold_type", "unknown")
                    # ★ 여기서 복원!
                    d = restore_and_merge(d, orig_data, p_mult, s_mult)
                    dfs.append(d)
            
            if not dfs: continue
            full_df = pd.concat(dfs).sort_values("timestamp").reset_index(drop=True)
            
            # Dev / Holdout 분리
            dev = full_df[full_df["fold_type"] != "final_holdout"]
            hold = full_df[full_df["fold_type"] == "final_holdout"]
            
            # 백테스트 실행
            dev_ret, dev_mdd = backtest_aggressive(dev, FEE_RATE, RISK_PER_TRADE, MAX_PYRAMIDING)
            hold_ret, hold_mdd = 0.0, 0.0
            if len(hold) > 0:
                hold_ret, hold_mdd = backtest_aggressive(hold, FEE_RATE, RISK_PER_TRADE, MAX_PYRAMIDING)
            
            results.append({
                "Trial": trial.name,
                "Model": model,
                "Dev_Ret": dev_ret, "Dev_MDD": dev_mdd,
                "Hold_Ret": hold_ret, "Hold_MDD": hold_mdd
            })
            
    # 결과 출력 및 저장
    if results:
        res_df = pd.DataFrame(results).sort_values("Hold_Ret", ascending=False)
        
        # 퍼센트 포맷팅
        print_df = res_df.copy()
        for c in ["Dev_Ret", "Dev_MDD", "Hold_Ret", "Hold_MDD"]:
            print_df[c] = print_df[c].apply(lambda x: f"{x*100:.2f}%")
            
        print("\n" + "="*60)
        print("TOP 10 AGGRESSIVE STRATEGIES (Restored Data)")
        print("="*60)
        print(print_df.head(10).to_string(index=False))
        
        res_df.to_csv(BASE_DIR / "sniper_restored_aggressive_final.csv", index=False)
        print(f"\nSaved to: {BASE_DIR / 'sniper_restored_aggressive_final.csv'}")
    else:
        print("No results found.")



1. Loading Original Data (eth_4hour.csv)...
   -> Success! ATR & Next_Open calculated.

2. Starting Aggressive Backtest (All Trials)...
   Risk: 2.0%, Max Pos: 5
   Processing T0_L12_P1.0_S0.5_N30...
   Processing T0_L3_P1.2_S0.6_N20...
   Processing T0_L9_P1.4_S0.9_N30...
   Processing T10_L9_P1.8_S1.0_N30...
   Processing T12_L9_P1.8_S0.6_N30...
   Processing T13_L6_P1.8_S1.0_N25...
   Processing T14_L9_P1.0_S0.7_N25...
   Processing T15_L12_P1.8_S0.5_N25...
   Processing T16_L12_P1.4_S0.6_N25...
   Processing T17_L12_P1.8_S0.7_N30...
   Processing T18_L6_P1.4_S0.9_N20...
   Processing T19_L12_P1.8_S0.5_N15...
   Processing T1_L3_P1.2_S0.5_N10...
   Processing T1_L9_P1.2_S0.7_N15...
   Processing T22_L9_P1.8_S0.5_N20...
   Processing T23_L6_P1.6_S0.6_N25...
   Processing T24_L9_P1.8_S0.5_N30...
   Processing T25_L9_P1.6_S0.6_N30...
   Processing T26_L12_P1.8_S0.5_N30...
   Processing T27_L12_P1.0_S0.7_N30...
   Processing T28_L12_P1.6_S0.5_N30...
   Processing T29_L12_P1.6_S0.9_N30..

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path
import json
import pandas_ta as ta  # ATR 계산용 (없으면 pip install pandas_ta)

# ============================================================
# [설정] 현물(Spot) 백테스트 환경
# ============================================================
BASE_DIR = Path("2025-11-27_Sniper")
# ★ 중요: 원본 4시간봉 데이터 경로 (파일명 확인 필수)
ORIGINAL_DATA_PATH = Path("../eth_4hour.csv") 

FEE_RATE = 0.0005       # 0.05% (업비트 기준)
INITIAL_EQUITY = 1.0    # 시작 자산 1.0 (100%)
MODELS = ["CatBoost", "XGBoost", "LightGBM", "RandomForest"]

# [현물 자금 관리 설정]
MAX_PYRAMIDING = 2      

# ============================================================
# [함수 1] 지표 계산 헬퍼
# ============================================================
def calculate_metrics(equity_curve):
    if len(equity_curve) < 2:
        return {"Return": 0.0, "MDD": 0.0, "Sharpe": 0.0}

    final_return = equity_curve.iloc[-1] / equity_curve.iloc[0] - 1.0
    
    roll_max = equity_curve.cummax()
    drawdown = equity_curve / roll_max - 1.0
    mdd = drawdown.min()
    
    pct_change = equity_curve.pct_change().dropna()
    if len(pct_change) > 1 and pct_change.std() > 0:
        sharpe = (pct_change.mean() / pct_change.std()) * np.sqrt(len(pct_change))
    else:
        sharpe = 0.0

    return {"Return": final_return, "MDD": mdd, "Sharpe": sharpe}

def calculate_bh_metrics_fold_cumulative(df, fee_rate=0.0005):
    """
    각 Fold별 B&H 수익률을 복리로 연결
    """
    if "close" not in df.columns:
        return {"Return": 0.0, "MDD": 0.0, "Sharpe": 0.0}

    bh_equity = [1.0]
    current_eq = 1.0
    df = df.sort_values("timestamp")
    
    for fold_name, group in df.groupby("fold"):
        group = group.sort_values("timestamp")
        if len(group) < 2: continue
        
        start_p = group.iloc[0]["close"]
        end_p = group.iloc[-1]["close"]
        
        raw_ret = (end_p - start_p) / start_p
        net_ret = (1 + raw_ret) * (1 - fee_rate)**2 - 1
        
        current_eq *= (1 + net_ret)
        bh_equity.append(current_eq)
        
    return calculate_metrics(pd.Series(bh_equity))


# ============================================================
# [함수 2] 데이터 복원 (Predictions + Original Price)
# ============================================================
def prepare_original_data(csv_path):
    """
    원본 4시간봉 데이터를 읽고, ATR 계산 및 필요한 컬럼 준비
    """
    if not csv_path.exists():
        # 현재 폴더 근처에서 찾아보기
        potentials = list(Path(".").rglob(csv_path.name))
        if potentials:
            csv_path = potentials[0]
        else:
            raise FileNotFoundError(f"원본 데이터가 없습니다: {csv_path}")
        
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.lower()
    
    date_col = 'timestamp' if 'timestamp' in df.columns else 'date'
    df[date_col] = pd.to_datetime(df[date_col])
    
    # 컬럼 찾기 (eth_high, high 등)
    h = df.get('high', df.get('eth_high'))
    l = df.get('low', df.get('eth_low'))
    c = df.get('close', df.get('eth_close'))
    o = df.get('open', df.get('eth_open'))
    
    # ATR 84 계산
    df['atr_84'] = ta.atr(h, l, c, length=84)
    
    # 다음 봉 시가 (진입가용)
    df['next_open'] = o.shift(-1)
    
    # ★ orig_close 명시적으로 생성
    df['orig_close'] = c
    
    # 필요한 컬럼만 리턴
    return df[[date_col, 'atr_84', 'next_open', 'orig_close']].rename(columns={date_col: 'timestamp'})

def restore_and_merge(preds_df, orig_data_df, profit_mult, stop_mult):
    """
    predictions.csv에 원본 데이터를 병합하여 가격정보 복원
    """
    df = preds_df.copy()
    if "timestamp" not in df.columns: return df
    df["timestamp"] = pd.to_datetime(df["timestamp"])
    
    # 1. 날짜 기준 병합
    df = pd.merge(df, orig_data_df, on="timestamp", how="left")
    
    # 2. 가격 복원 (real_entry_price)
    if "real_entry_price" not in df.columns:
        # next_open 우선 사용, 없으면 orig_close, 그것도 없으면 내부 close 사용
        fallback_close = df.get("orig_close", df.get("close"))
        df["real_entry_price"] = df["next_open"].fillna(fallback_close)
        
    # 3. TP/SL 복원
    if "take_profit_price" not in df.columns:
        atr = df["atr_84"].fillna(0)
        entry = df["real_entry_price"]
        df["take_profit_price"] = entry + (atr * profit_mult)
        df["stop_loss_price"] = entry - (atr * stop_mult)
        
    return df


# ============================================================
# [함수 3] 현물(Spot) 전용 백테스트 로직 (핵심)
# ============================================================
def backtest_spot_pyramiding(df, fee_rate, max_pos):
    """
    레버리지 없이, 자산을 max_pos 등분하여 진입하는 현물 전략
    """
    df = df.sort_values("timestamp").reset_index(drop=True)
    
    cash = INITIAL_EQUITY      # 보유 현금
    positions = []             # 보유 코인 리스트
    equity_curve = [INITIAL_EQUITY]
    
    for i, row in df.iterrows():
        curr_price = row["real_entry_price"] # 현재가 (시가)
        if pd.isna(curr_price) or curr_price <= 0:
            equity_curve.append(equity_curve[-1])
            continue

        # 1. 포지션 관리 (매도 체크)
        active_pos = []
        for pos in positions:
            # 익절 (TP)
            if curr_price >= pos['tp']:
                # 보수적 체결: 갭상승 시에도 TP 가격 체결 가정 (현실은 시가일 수 있음)
                # 여기서는 '시가가 TP를 넘었으면 시가 체결'로 조금 더 현실 반영
                sell_price = max(curr_price, pos['tp']) 
                sell_amt = pos['amount'] * sell_price
                fee = sell_amt * fee_rate
                cash += (sell_amt - fee)
                continue
                
            # 손절 (SL)
            if curr_price <= pos['sl']:
                sell_price = min(curr_price, pos['sl'])
                sell_amt = pos['amount'] * sell_price
                fee = sell_amt * fee_rate
                cash += (sell_amt - fee)
                continue
                
            active_pos.append(pos)
        positions = active_pos
        
        # 2. 신규 진입 (매수)
        sig = int(row.get("signal", 0))
        
        # 조건: 신호 발생 & 포지션 슬롯 남음
        if sig == 1 and len(positions) < max_pos:
            entry = float(row["real_entry_price"])
            sl = float(row["stop_loss_price"])
            tp = float(row["take_profit_price"])
            
            if entry > 0 and sl > 0:
                # 현재 총 자산 가치
                current_total_equity = cash + sum([p['amount']*curr_price for p in positions])
                
                # 1회 진입 목표 금액 (자산의 1/N)
                target_invest = current_total_equity / max_pos
                
                # 현금이 충분한가? (수수료 고려 99% 수준)
                if cash >= target_invest * 0.99:
                    buy_amt = target_invest
                    fee = buy_amt * fee_rate
                    real_buy_amt = buy_amt - fee 
                    amount = real_buy_amt / entry # 코인 개수
                    
                    cash -= buy_amt
                    positions.append({'entry': entry, 'tp': tp, 'sl': sl, 'amount': amount})

        # 3. 자산 평가 (Mark-to-Market)
        coin_val = sum([p['amount'] * curr_price for p in positions])
        total_equity = cash + coin_val
        equity_curve.append(total_equity)

    return calculate_metrics(pd.Series(equity_curve))


# ============================================================
# [Main] 전체 실행
# ============================================================
def load_preds(trial_dir, model_name):
    dfs = []
    for sub in trial_dir.iterdir():
        if not sub.is_dir(): continue
        meta_path = sub / "metadata.json"
        if not meta_path.exists(): continue
        with open(meta_path) as f: meta = json.load(f)
        
        p_path = sub / f"predictions_{model_name}.csv"
        if not p_path.exists(): continue
        
        df = pd.read_csv(p_path)
        if "timestamp" not in df.columns: df = df.reset_index().rename(columns={"index":"timestamp"})
        df["fold"] = sub.name
        df["fold_type"] = meta.get("fold_type", "unknown")
        dfs.append(df)
    
    if not dfs: return None
    return pd.concat(dfs).sort_values("timestamp").reset_index(drop=True)

if __name__ == "__main__":
    print("1. Loading Original Data (eth_4hour.csv)...")
    try:
        orig_data = prepare_original_data(ORIGINAL_DATA_PATH)
        print("   -> Success! Data ready for restoration.")
    except Exception as e:
        print(f"   -> Error: {e}")
        exit()

    dev_results = []
    hold_results = []
    
    print(f"\n2. Starting SPOT Backtest (Max {MAX_PYRAMIDING} Pos)...")

    for trial in sorted(BASE_DIR.iterdir()):
        if not trial.is_dir() or not trial.name.startswith("T"): continue
        
        try:
            with open(next(trial.glob("*/metadata.json"))) as f:
                params = json.load(f).get("trial_params", {})
            p_mult = float(params.get("profit_mult", 1.6))
            s_mult = float(params.get("stop_mult", 0.5))
        except: continue

        print(f"   Processing {trial.name}...")

        for model in MODELS:
            # 데이터 로드
            df = load_preds(trial, model)
            if df is None: continue
            
            # 복원
            df = restore_and_merge(df, orig_data, p_mult, s_mult)
            
            # 분리
            dev_df = df[df["fold_type"] != "final_holdout"]
            hold_df = df[df["fold_type"] == "final_holdout"]
            
            # 백테스트 (현물 피라미딩)
            dev_strat = backtest_spot_pyramiding(dev_df, FEE_RATE, MAX_PYRAMIDING)
            dev_bh = calculate_bh_metrics_fold_cumulative(dev_df, FEE_RATE)
            
            dev_results.append({
                "Trial": trial.name, "Model": model,
                "Strat_Ret": dev_strat["Return"], "BH_Ret": dev_bh["Return"],
                "Strat_MDD": dev_strat["MDD"], "BH_MDD": dev_bh["MDD"],
                "Strat_Sharpe": dev_strat["Sharpe"], "BH_Sharpe": dev_bh["Sharpe"]
            })
            
            if len(hold_df) > 0:
                hold_strat = backtest_spot_pyramiding(hold_df, FEE_RATE, MAX_PYRAMIDING)
                hold_bh = calculate_bh_metrics_fold_cumulative(hold_df, FEE_RATE)
                
                hold_results.append({
                    "Trial": trial.name, "Model": model,
                    "Strat_Ret": hold_strat["Return"], "BH_Ret": hold_bh["Return"],
                    "Strat_MDD": hold_strat["MDD"], "BH_MDD": hold_bh["MDD"],
                    "Strat_Sharpe": hold_strat["Sharpe"], "BH_Sharpe": hold_bh["Sharpe"]
                })

    # ============================================================
    # [출력]
    # ============================================================
    pd.set_option('display.max_rows', 20)
    
    def format_df(df):
        out = df.copy()
        pct_cols = [c for c in out.columns if "Ret" in c or "MDD" in c]
        for c in pct_cols:
            out[c] = out[c].apply(lambda x: f"{x*100:.2f}%")
        return out

    print("\n" + "="*100)
    print("[ANALYSIS 1] Walk-Forward (Dev) - SPOT Strategy vs B&H")
    print("="*100)
    if dev_results:
        df_dev = pd.DataFrame(dev_results).sort_values("Strat_Ret", ascending=False)
        print(format_df(df_dev).head(10).to_string(index=False))
        df_dev.to_csv(BASE_DIR / "summary_spot_dev.csv", index=False)

    print("\n" + "="*100)
    print("[ANALYSIS 2] Final Holdout - SPOT Strategy vs B&H")
    print("="*100)
    if hold_results:
        df_hold = pd.DataFrame(hold_results).sort_values("Strat_Ret", ascending=False)
        print(format_df(df_hold).head(10).to_string(index=False))
        df_hold.to_csv(BASE_DIR / "summary_spot_holdout.csv", index=False)

    print(f"\nFiles saved to {BASE_DIR}")



1. Loading Original Data (eth_4hour.csv)...
   -> Success! Data ready for restoration.

2. Starting SPOT Backtest (Max 2 Pos)...
   Processing T0_L12_P1.0_S0.5_N30...
   Processing T0_L3_P1.2_S0.6_N20...
   Processing T0_L9_P1.4_S0.9_N30...
   Processing T10_L9_P1.8_S1.0_N30...
   Processing T12_L9_P1.8_S0.6_N30...
   Processing T13_L6_P1.8_S1.0_N25...
   Processing T14_L9_P1.0_S0.7_N25...
   Processing T15_L12_P1.8_S0.5_N25...
   Processing T16_L12_P1.4_S0.6_N25...
   Processing T17_L12_P1.8_S0.7_N30...
   Processing T18_L6_P1.4_S0.9_N20...
   Processing T19_L12_P1.8_S0.5_N15...
   Processing T1_L3_P1.2_S0.5_N10...
   Processing T1_L9_P1.2_S0.7_N15...
   Processing T22_L9_P1.8_S0.5_N20...
   Processing T23_L6_P1.6_S0.6_N25...
   Processing T24_L9_P1.8_S0.5_N30...
   Processing T25_L9_P1.6_S0.6_N30...
   Processing T26_L12_P1.8_S0.5_N30...
   Processing T27_L12_P1.0_S0.7_N30...
   Processing T28_L12_P1.6_S0.5_N30...
   Processing T29_L12_P1.6_S0.9_N30...
   Processing T2_L3_P1.6_S0.8_N